<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## mmlupro_philosophy TextBook dataset

In [1]:
# set parameters
 
file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()
file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [2]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")
# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 1. build DB (using Wikipida)

### what is the main topic & specific topic of query

In [3]:
# read mmlu_pro.csv file
import pandas as pd
import os

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

prompts, answers = read_data(os.path.join(data_path, 'mmlupro_test_philosophy.csv'))

In [4]:
testdata = pd.read_csv(data_path+'mmlupro_test_philosophy.csv')
testdata

,prompts,answers
0,QUESTION0) Select the best translation into pr...,A
1,QUESTION1) Construct a complete truth table f...,D
2,QUESTION2) Which of the following proposition...,I
3,QUESTION3) Select the best translation into pr...,A
4,QUESTION4) Select the best translation into pr...,E
...,...,...
494,QUESTION494) Mill claims that in deciding how ...,C
495,QUESTION495) Of the two versions of the princi...,A
496,"QUESTION496) According to Kant, laws of nature...",C
497,QUESTION497) Hare refers to people who endorse...,A


In [5]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3'])

for index, row in testdata.iterrows():
    #if index % 10 != 0 : continue # 일단 실험할 땐 개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]

    try : 
        embedded_query = passage_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a}
    except :
        print(f'pass: {index}')
        continue


In [6]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3
0,0,"[-0.00475311279296875, 0.00991058349609375, -0...",Select the best translation into predicate log...,Select the best translation into predicate log...,A,None,None,None
1,1,"[0.0004830360412597656, -0.0250091552734375, -...",Construct a complete truth table for the foll...,Construct a complete truth table for the foll...,D,None,None,None
2,2,"[0.0006518363952636719, 0.004669189453125, -0....",Which of the following propositions is an imm...,Which of the following propositions is an imm...,I,None,None,None
3,3,"[-0.002498626708984375, -0.007579803466796875,...",Select the best translation into predicate log...,Select the best translation into predicate log...,A,None,None,None
4,4,"[-0.0167694091796875, -0.007282257080078125, -...",Select the best translation into predicate log...,Select the best translation into predicate log...,E,None,None,None
...,...,...,...,...,...,...,...,...
494,494,"[-0.0107269287109375, 0.0018281936645507812, -...",Mill claims that in deciding how to act:\n,Mill claims that in deciding how to act:\n(A) ...,C,None,None,None
495,495,"[-0.00873565673828125, -0.0032100677490234375,...",Of the two versions of the principle that Sing...,Of the two versions of the principle that Sing...,A,None,None,None
496,496,"[-0.004878997802734375, -0.0228729248046875, -...","According to Kant, laws of nature are laws acc...","According to Kant, laws of nature are laws acc...",C,None,None,None
497,497,"[-0.0151214599609375, -0.0167999267578125, -0....",Hare refers to people who endorse ideals witho...,Hare refers to people who endorse ideals witho...,A,None,None,None


In [7]:
nowtest.question[1]

' Construct a complete truth table for the following pairs of propositions. Then, using the truth tables, determine whether the statements are logically equivalent or contradictory. If neither, determine whether they are consistent or inconsistent. Justify your answers.\n(~M ⊃ ~N) ∨ (O ≡ N) and (~M · N) · [(~O ∨ ~N) · (O ∨ N)]\n'

# DB 불러오기

In [8]:
import numpy as np

# .npy 파일 로드 (타입==넘파이)
textbookDB = np.load(data_path+f'embedding/full_philosophy_textbook.npy')
textbookDB = textbookDB.tolist()

textbookDB_embed = np.load(data_path+f'embedding/full_philosophy_textbook_embed.npy')
textbookDB_embed = textbookDB_embed.tolist()

In [9]:
textbookDB[0]

'of morality and of value generally. These close\nconnections are no insignificant part of what gives the disputes their\nimportance to the traditions involved. Discerning these traditions\nwidens one’s understanding of the views involved, and enriches the\nsorts of possible assessments of competing appeals to experience.\nClosely connected with these topics are competing notions of human\nsurvival of death and whether any of them have any basis or support.SummaryThe core of philosophy of religion, as of philosophy generally, is\nmetaphysics and epistemology, systematic attempts to give defensible\nanswers to the questions What is there? and How can we know what\nthere is? At the core of any religious tradition is its own answer to\nthese questions, construed as and embedded in an answer to the basic\nproblem to which the tradition addresses itself as the rationale for its\nexistence. Thus our own concentration on accounts of religious reality\nand religious knowledge. How is ultimate 

# 유사도 계산 -> sorting

In [10]:
import numpy as np

for idx, row in nowtest.iterrows() : # 질문 받아오기 
    #if idx == 50 : break

    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(textbookDB_embed).T).argsort()[::-1]

    nowtest.at[idx, 'top1'] = textbookDB[sorted_idx[0]]
    nowtest.at[idx, 'top2'] = textbookDB[sorted_idx[1]]
    nowtest.at[idx, 'top3'] = textbookDB[sorted_idx[2]]


In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    1. Use the information from the [context] to find the correct answer [answer] to the [question].
    2. Please provide most correct answer.
    
    3. When returning the answer, DO NOT exlain anything. And you must also include the choice number like :
    Answer : (Number) the answer choice
    ---
    
    Question: {question}
    Context: {context1}
    Answer :
    ---
        
    '''

)
zs_chain = prompt_template | llm

responses = []
for idx, row in nowtest.iterrows() :
    if idx == 100 : break
        
    response = zs_chain.invoke({"question": row.prompts, "context1": row.top1}) # 선지 전까지 받아오기
    responses.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

responses


['Answer : (A) Tdc',
 'Answer : (E) Neither logically equivalent nor contradictory, but inconsistent',
 'Answer: (I) ~(X ≡ ~Y)',
 'Answer : (A) Lhf',
 'Answer : (D) ~eBj',
 'Answer : (I) (∀x)[Bx ⊃ (∃y)(Ny • Lxy)]',
 'Answer: (C) Invalid. Counterexample when L, N, O, Q, and R are true and M and P are false',
 'It is wrong for society to kill a murderer.',
 'Answer: (J) the refusal to execute criminals has a civilizing effect.',
 'Answer : (D) having to choose between addressing immediate harm and addressing structural injustice.',
 'Answer : (B) are flawed due to lack of empirical evidence.',
 'Answer: (B) to argue for the potential benefits of cloning',
 'Answer: (F) creating a global consensus on acceptable genetic engineering practices',
 'Answer: (B) the non-instrumental value of caring actions as manifestations of intrinsic goods for person like love and forgiveness.',
 'Answer: (B) humanitarian',
 'Answer: (I) people are allowed to do a smaller evil to prevent a greater one.',
 'A

In [ ]:
nowtest

In [18]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Please provide most correct answer. Let's think step by step.
    
    When translating the answer, DO NOT exlain anything. And you must also include the choice number like :
    Answer : (Number) the answer choice
    ---
    
    Question: {question}
    Context: {context1} {context2}
    Answer :
    ---
        
    '''

)
zs_chain2 = prompt_template | llm


responses2 = []
for idx, row in nowtest.iterrows() :
    if idx == 70 : break

        
    response = zs_chain2.invoke({"question": row.prompts, "context1": row.top1, "context2": row.top2}) # 선지 전까지 받아오기
    responses2.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

responses2

RateLimitError: Error code: 429 - {'error': {'message': 'You have reached API request limit. Please wait and try again later. If your use-case require a higher rate limit, please request at https://support.upstage.ai', 'type': 'too_many_requests', 'param': '', 'code': 'too_many_requests'}}

## Check Accuracy

In [14]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

try : del [[wrongdf]]
except : pass
wrongdf = pd.DataFrame(columns=['questionNum', 'answer', '1st_pred'])

# print accuracy
cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, responses2)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":"-"}
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":None}
        
acc = cnt/100*100
print(f"acc: {acc}%")
print()
print("1st wrong:", wrong)
wrongdf.loc[len(wrongdf)] = {"questionNum":"-", "answer" : "-", "1st_pred":acc}

----------
Answer : (A) Tdc
idx: 0 | generated answer: A, answer: A
----------
Answer : (E) Neither logically equivalent nor contradictory, but inconsistent
idx: 1 | generated answer: E, answer: D
----------
Answer: (I) ~(X ≡ ~Y)
idx: 2 | generated answer: I, answer: I
----------
Answer : (A) Lhf
idx: 3 | generated answer: A, answer: A
----------
Answer : (D) ~eBj
idx: 4 | generated answer: D, answer: E
----------
Answer : (I) (∀x)[Bx ⊃ (∃y)(Ny • Lxy)]
idx: 5 | generated answer: I, answer: I
----------
Answer: (C) Invalid. Counterexample when L, N, O, Q, and R are true and M and P are false
idx: 6 | generated answer: C, answer: D
----------
It is wrong for society to kill a murderer.
extraction fail
----------
Answer: (J) the refusal to execute criminals has a civilizing effect.
idx: 8 | generated answer: J, answer: D
----------
Answer : (D) having to choose between addressing immediate harm and addressing structural injustice.
idx: 9 | generated answer: D, answer: H
----------
Answer 

In [16]:
wrongdf

,questionNum,answer,1st_pred
0,2,D,E
1,5,E,D
2,7,D,C
3,8,J,None
4,9,D,J
...,...,...,...
57,93,B,E
58,95,C,D
59,96,D,J
60,99,I,D
